## Read GNSS Data into Portal/AGOL

Using the ArcGIS Python API
Read a NMEA file from a URL, parse the $GPGGA line which contains the lat/long and other GPS attributes.

[ArcGIS Python API - Guide]("https://developers.arcgis.com/python/")

[ArcGIS Python API - Reference]("https://developers.arcgis.com/python/api-reference/")

In [24]:
import re, shutil, tempfile
from urllib.request import urlopen
import pandas as pd
from arcgis.gis import GIS

gis = GIS("https://krennic.esri.com/portal", "admin", "esri.agp")

#### Convert NMEA coords to lat/long
```
DD = int(float(n)/100)
SS = float(n) - DD * 100
coordinate = DD + SS / 60
```

In [2]:
def nmea_to_latlong(val):
    dd = int(float(val)/100)
    ss = float(val) - dd * 100
    return dd + ss / 60

#### Download or stream NMEA data, parse the GPGGA sentence to get coords and altitude
The UTC position value could be helpful for determining the order of the points.

[Generate some NMEA data](https://nmeagen.org/)

In [26]:
## Provide a url of nmea data then pass tmp_file to the next open function...
with urlopen("https://raw.githubusercontent.com/dotMorten/NmeaParser/master/src/NmeaSampleData.txt") as response:
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        shutil.copyfileobj(response, tmp_file)
        
with open(tmp_file.name) as log:
    gngns = []
    for line in log:
        if line.startswith("$GPGGA"):
            gngns.append(line.split(","))
    pts = []
    for i, val in enumerate(gngns):
        lat = nmea_to_latlong(val[2])
        long = nmea_to_latlong(val[4])
        if val[5] == "W":
            long = long - (long * 2)
        if val[3] == "S":
            lat = lat - (lat * 2)
        pts.append(
            {"utc_pos": int(float(val[1])), 
             "latitude": nmea_to_latlong(val[2]), 
             "longitude": long, 
             "altitude": float(val[9])})
    
    
    # Write data to a csv file.
    df = pd.DataFrame(pts)
    df.index.name = 'objid'
    df.astype({"utc_pos": "int32"})
    df.to_csv("coords.csv")
    
# Print some rows to verify
df.loc[df.index < 5, ["latitude", "longitude"]]

{'utc_pos': 235234, 'latitude': 39.43283666666667, 'longitude': -119.76559, 'altitude': 1378.6}


,latitude,longitude
objid,,
0,39.432837,-119.765590
1,39.432465,-119.765352
2,39.432095,-119.765113
3,39.431725,-119.764875
4,39.431358,-119.764637


#### Publish to Portal

This example is using ArcGIS Enterprise but this will also work with ArcGIS Online.  Just look up how to authenticate.
[ArcGIS Python API - Publish to Portal]("https://developers.arcgis.com/python/guide/accessing-and-creating-content/")

In [25]:
pts_properties = {
    'title': 'GNSS Data Feed',
    'tags': 'GNSS, GPS, Python',
    'type': 'CSV'
}

coords_csv = gis.content.add(pts_properties, data="coords.csv")
coords_csv.publish()

<Item title:"GNSS Data Feed" type:Feature Layer Collection owner:admin>